In [1]:
import pandas as pd
import numpy as np
import math
import sys
import random
from pivottablejs import pivot_ui
import sklearn
from random import sample
from sklearn.neighbors import NearestNeighbors
import json

In [2]:
# Pearson Score
# ------------------------------------------------------------------------------------------------------------------
def pearson_calc(numbers_x, numbers_y):
    mean_x = 0.000
    mean_y = 0.000
    if(len(numbers_y)==0 and len(numbers_x)==0):
        mean_x = sum(numbers_x)/0.001
        mean_y = sum(numbers_y)/0.001
    elif(len(numbers_x)==0):
        mean_x = sum(numbers_x)/0.001
        mean_y = sum(numbers_y)/len(numbers_y)
    elif(len(numbers_y)==0):
        mean_x = sum(numbers_x)/len(numbers_x)
        mean_y = sum(numbers_y)/0.001
    else:
        mean_x = sum(numbers_x)/len(numbers_x)
        mean_y = sum(numbers_y)/len(numbers_y)
    
    subtracted_mean_x = [i - mean_x for i in numbers_x]
    subtracted_mean_y = [i - mean_y for i in numbers_y]

    x_times_y = [a * b for a, b in list(zip(subtracted_mean_x, subtracted_mean_y))]

    x_squared = [i * i for i in numbers_x]
    y_squared = [i * i for i in numbers_y]
    
    N = sum(x_times_y)
    D = math.sqrt(sum(x_squared) * sum(y_squared))
    if (D !=0):
        return N/D
    else:
        return N/0.001

In [3]:
def match_cal(user1,user2):
    match = 0
    df_u1 = user1.values.tolist()
    df_u2 = user2.values.tolist()
    df_u1 =  [y for x in df_u1 for y in x] 
    df_u2 =  [y for x in df_u2 for y in x] 
    for i in range(0,len(df_u2)):
        if(df_u2[i]==1 and df_u1[i]==1):
            match=match+1
        else:
            match=match+0.001
#     print('match',match)
    pearson_correlation_val = pearson_calc(df_u1,df_u2)
    return match,pearson_correlation_val

In [4]:
def pearson_score(user1,user2):
    '''
    user1 & user2 : user ids of two users between which similarity score is to be calculated.
    '''
    # A list of categories joined by both the users.
    both_match_count, score = match_cal(user1,user2)
    
#     print('both matches :',both_match_count)
    # Returning '0' if no common values.
    if both_match_count == 0 :
        return 0
    return score

In [5]:
def find_k_neighbours(df,k):
    order = np.argsort(df.values, axis=1)[:, :k]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:k].index, index=['top{}'.format(i) for i in range(1, k+1)]), axis=1)
    return df

In [6]:
def weighted_average_over_scores(score,weights):
    L = len(score)
    weighted_sum = 0.000
    values_weighted_sum = 0.000
    weight_list_gen = []
    for i in range(0,L):
        total_weight = score[i] * weights[i]
#         print('total_weight :',total_weight)
        weight_list_gen.append(total_weight)
    for i in range(L):
#         weighted_sum = weighted_sum + weight_list_random_gen[i - 1]
        weighted_sum = weighted_sum + weight_list_gen[i - 1]
    for i in range(L):
        weight_list_gen.append(score[i - 1] * weight_list_gen[i - 1])
    for i in range(len(weight_list_gen)):
        values_weighted_sum = values_weighted_sum + weight_list_gen[i - 1]
#     if(weighted_sum == 0):
    if(int(weighted_sum) ==0):
#         print("Something is wrong!")
        return values_weighted_sum / 0.001
    else:
        return values_weighted_sum / weighted_sum

In [7]:
def gbellmf(x):
    return 1.00 / (1.00 + np.abs((x - 0.001) / 0.001) ** (2 * 0.001))

In [8]:
# ------------------------------------------------------------------------------------------------------------------

In [9]:
data_users = pd.read_csv('user_cat.csv',index_col=0)
data_users.head()

C:\Users\tusha\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,member_id,category_id,group_id,rating
0,3,2,490552,4.67
1,210,2,490552,4.67
2,357518,2,490552,4.67
3,511008,2,490552,4.67
4,915553,2,490552,4.67


In [10]:
data_users.size

12511788

In [11]:
data_users.shape

(3127947, 4)

In [12]:
tester = data_users.sample(n=1000).copy()
tester.head()

,member_id,category_id,group_id,rating
1898475,204966818,2,6896742,4.38
244396,200314454,34,19329611,4.89
1876595,27478342,12,1770593,4.47
2604662,190658622,31,12685042,4.50
175166,49529212,34,1235063,4.83


In [13]:
tester.to_csv("tester_subset.csv")

In [14]:
tester.shape

(1000, 4)

In [15]:
# Creating a pivot table based on above random subset of data

In [16]:
tester['values']=1

In [17]:
# Checking if the dataset for values
tester.head()

,member_id,category_id,group_id,rating,values
1898475,204966818,2,6896742,4.38,1
244396,200314454,34,19329611,4.89,1
1876595,27478342,12,1770593,4.47,1
2604662,190658622,31,12685042,4.50,1
175166,49529212,34,1235063,4.83,1


In [18]:
df_pivot = tester.pivot_table(index=['member_id'], columns=['category_id'],values='values').fillna(0)
df_pivot.head()

category_id,1,2,4,5,6,8,9,10,11,12,...,26,27,28,29,30,31,32,33,34,36
member_id,,,,,,,,,,,,,,,,,,,,,
98396,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
363596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
729488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1292867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1324145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [19]:
# pivot_ui(df_pivot,rows=['member_id'], cols=['category_id'],outfile_path="./data_json/members_category_pivot.html")

In [20]:
# Shape of the pivot table
df_pivot.shape

(994, 31)

In [21]:
score_per_user_match = []
member_vals = []
index_list = []
for index,row in df_pivot.iterrows():
    index_list.append(index)

In [22]:
# Displaying all the members in the above pivot table
print(len(index_list))

994


In [23]:
scores = []
members1 = []
members2 = []
result = []
for i in range(0,len(index_list)):
#     print('user 1 --',user1)
    members1.append(index_list[i])
    user1 = df_pivot.iloc[i:i+1]
    for j in range(0,len(index_list)):
        user2 = df_pivot.iloc[j:j+1]
#         print('user 2 --',user2)
        score = pearson_score(user1,user2)
        scores.append(score)
        members2.append(index_list[j])
        list_add = []
        list_add.append(index_list[i])
        list_add.append(index_list[j])
        list_add.append(score)
        result.append(list_add)

In [24]:
checking = pd.DataFrame(result, columns = ['users_A', 'users_B','scores']) 
checking.head()

,users_A,users_B,scores
0,98396,98396,0.967742
1,98396,363596,-0.032258
2,98396,729488,-0.032258
3,98396,1292867,-0.032258
4,98396,1324145,-0.032258


In [25]:
df_pivot_new=checking.pivot_table(index=['users_A'], columns=['users_B'], values = 'scores').fillna(0)
df_pivot_new.head()

users_B,98396,363596,729488,1292867,1324145,1475155,1762810,2108135,2136899,2144070,...,238621714,239191374,239573671,239633192,239716243,240189188,240351164,240443938,240599389,240776067
users_A,,,,,,,,,,,,,,,,,,,,,
98396,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,...,-0.032258,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258
363596,-0.032258,0.967742,0.967742,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,0.967742,-0.032258,...,-0.032258,-0.032258,0.967742,-0.032258,0.967742,-0.032258,0.967742,-0.032258,-0.032258,0.967742
729488,-0.032258,0.967742,0.967742,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,0.967742,-0.032258,...,-0.032258,-0.032258,0.967742,-0.032258,0.967742,-0.032258,0.967742,-0.032258,-0.032258,0.967742
1292867,-0.032258,0.967742,0.967742,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,0.967742,-0.032258,...,-0.032258,-0.032258,0.967742,-0.032258,0.967742,-0.032258,0.967742,-0.032258,-0.032258,0.967742
1324145,-0.032258,-0.032258,-0.032258,-0.032258,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,...,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258


In [26]:
# Finding the 3 best suited neighboours for each member in the pivot table which is our user-user matrix

In [27]:
df_pivot_new.to_json(r'./data_json/user_user_matrix_normal.json')

In [28]:
df_pivot_new.to_csv("user_user_matrix_normal.csv")

In [29]:
# pivot_ui(df_pivot_new,rows=['users_A'], cols=['users_B'],outfile_path="./data_json/user_user_pivot.html")

In [30]:
neigh_3 = find_k_neighbours(df_pivot_new,5)
neigh_3.head()

,top1,top2,top3,top4,top5
users_A,,,,,
98396,98396,34003912,7836194,187115258,202991217
363596,240776067,194714163,49965992,49529212,196838261
729488,240776067,194714163,49965992,49529212,196838261
1292867,240776067,194714163,49965992,49529212,196838261
1324145,192577002,161782652,45921342,45897912,14578242


In [31]:
user_list = list(neigh_3.index)
user_list = sample(user_list, 5)
print(user_list)

[232768376, 184733438, 182508463, 14514615, 22982331]


<b> Converting the User-List to JSON

In [71]:
# with open('./data_json/user_list_all_selected_members.json', 'w') as json_file:
#     json.dump(user_list, json_file)

<b> Converting neirest neigbour matrix to json

In [33]:
neigh_3.to_json(r'./data_json/neigh_3.json')

In [34]:
neigh_3.to_csv("neigh_3.csv")

In [35]:
# Finding unique members from K-NN algorithm

In [64]:
# neigh_3.loc[232768376]

In [65]:
# test = pd.read_csv('neigh_3.csv')
# test=test.set_index('users_A')
# test

In [66]:
# test.loc[232768376]

In [67]:
def unique_mem(selected_user):
    test = pd.read_csv('neigh_3.csv')
    test=test.set_index('users_A')
    selected_users_check = []
    neigh_3 = test.loc[selected_user]
    for i in neigh_3:
        if(i not in selected_users_check and (i is not None)):
            selected_users_check.append(i)
    member_list=[]
    for i in range(0,len(selected_users_check)):
        if(selected_user == selected_users_check[i]):
            pass
        else:
            member_list.append(selected_users_check[i])
    return member_list

In [68]:
# Looping through all the users

In [69]:
for l in range(0,len(user_list)):
    
    # tester_new = pd.read_csv("tester_subset.csv")
    
    # GEtting the user from the "USER_LIST"
    selected_user = user_list[l]
    
    user_val = df_pivot.loc[selected_user:selected_user]
    
    
    member_list = unique_mem(selected_user)
    
    # copy group and filter members based on above data
    copy_data = data_users.copy()
    filter_groups_mem_info = copy_data['member_id'].isin(member_list)
    copy_data.where(filter_groups_mem_info,inplace=True)
    copy_data = copy_data.dropna()
    
    # Getting the group infromatin from based on the above data
    group_list_vals = copy_data['group_id'].to_list()
    group_list_vals = set(group_list_vals)
    group_list_vals = list(group_list_vals)
    
    
    random_5_groups = sample(group_list_vals,10)
    groups_all = data_users.copy()
    
    filter_groups_from_rand = groups_all['group_id'].isin(random_5_groups)
    groups_all.where(filter_groups_from_rand,inplace=True)
    groups_all = groups_all.dropna()
    
    weighted_score = []
    result_groups = []
    members1 = []
    members2 = []
    scores = []
    weights = []
    groups_to_check = []
    
    for i in range(0,len(random_5_groups)):
        group_value = random_5_groups[i]
        data_users_loop = pd.read_csv('user_cat.csv',index_col=0)
        filer_loop = data_users_loop['group_id'] == group_value
        data_users_loop = data_users_loop.dropna()
        data_users_loop

        # Converting the above dataframe to pivot table - User - Category Matrix
        data_users_loop['values']=1
        df_pivot_new = data_users_loop.pivot_table(index=['member_id'], columns=['category_id'], values = 'values').fillna(0)

        score_per_user_match = []
        member_vals = []
        index_list = []

        # Fetch all the members of this group
        for index,row in df_pivot_new.iterrows():
            index_list.append(index)

        # Iterating over the fetched users
        user1 = user_val


        for j in range(0,len(index_list)):
            user2 = df_pivot.iloc[j:j+1]
            # print('user 2 --',user2)

            score = pearson_score(user1,user2)
            # print("Score : ",score)

            scores.append(score)

            weights.append(gbellmf(score))    

            members1.append(user1)
            members2.append(index_list[j])

            list_add = []
            list_add.append(user1)
            list_add.append(index_list[j])
            # score_list_old.append(score)
            list_add.append(score)
            result_groups.append(list_add)

            # Finding the weighted average
            # print("scores :",scores)
            # print("weights :",weights)

        update = weighted_average_over_scores(scores,weights)
        weighted_score.append(update)
        
    Threshold_val = np.mean(weighted_score)
    user_value_selected = list(user_val.index)
    user_value_selected = user_value_selected[0]
    
    final_result = []
    for i in range(0,len(weighted_score)):
        if(weighted_score[i]>=Threshold_val):
            list_final = []
            list_final.append(user_value_selected)
            list_final.append(random_5_groups[i])
            list_final.append(weighted_score[i])

            final_result.append(list_final)
    
    final_df = pd.DataFrame(final_result, columns = ['users_A', 'groups','scores']) 
    df = final_df.pivot_table(index=['users_A'], columns=['groups'], values = 'scores').fillna(0)
    df.to_csv('result'+ '_'+ str(selected_user) +'.csv')
    
    select_memebrs = pd.read_csv("members.csv", encoding = "Latin-1")
    filter_m = select_memebrs['member_id'] == selected_user
    select_memebrs.where(filter_m,inplace=True)
    select_memebrs = select_memebrs.dropna()
    select_memebrs = select_memebrs[['member_id', 'group_id']]
    select_memebrs.to_csv('all_data'+'_' +  str(selected_user) +'.csv')

In [72]:
data_all_user_list = pd.read_csv('members.csv', encoding = "latin-1")
filter_user_list = data_all_user_list['member_id'].isin(user_list)
data_all_user_list.where(filter_user_list,inplace=True)
data_all_user_list = data_all_user_list.dropna()
data_all_user_list.head()

,member_id,bio,city,country,hometown,joined,lat,link,lon,member_name,state,member_status,visited,group_id
1148163,14514615.0,not_found,New York,us,not_found,2011-12-29 02:14:14,40.79,http://www.meetup.com/members/14514615,-73.95,Abel M,NY,active,2016-04-05 00:48:11,107592.0
1148164,14514615.0,not_found,New York,us,not_found,2011-04-17 13:36:49,40.79,http://www.meetup.com/members/14514615,-73.95,Abel M,NY,active,2014-06-06 16:59:27,1209438.0
1148165,14514615.0,not_found,New York,us,not_found,2011-04-17 13:38:51,40.79,http://www.meetup.com/members/14514615,-73.95,Abel M,NY,active,2016-05-17 03:08:39,1228907.0
1148166,14514615.0,not_found,New York,us,not_found,2011-04-17 14:06:05,40.79,http://www.meetup.com/members/14514615,-73.95,Abel M,NY,active,2012-08-13 16:49:04,1290254.0
1148167,14514615.0,not_found,New York,us,not_found,2016-01-20 20:51:09,40.79,http://www.meetup.com/members/14514615,-73.95,Abel M,NY,active,2016-02-24 23:00:50,1428523.0


In [73]:
data_all_user_list.shape

(138, 14)

In [74]:
# data_all_user_list.to_csv("data_all_user_list.csv")

In [75]:
groups_info = pd.read_csv('groups.csv', encoding="Latin-1")
merged_data = pd.merge(data_all_user_list,groups_info,on="group_id")
merged_data.head()

,member_id,bio,city_x,country_x,hometown,joined,lat_x,link_x,lon_x,member_name,...,organizer.photo.photo_link,organizer.photo.thumb_link,organizer.photo.type,rating,state_y,timezone,urlname,utc_offset,visibility,who
0,14514615.0,not_found,New York,us,not_found,2011-12-29 02:14:14,40.79,http://www.meetup.com/members/14514615,-73.95,Abel M,...,https://secure.meetupstatic.com/photos/member/...,https://secure.meetupstatic.com/photos/member/...,member,4.51,NY,US/Eastern,mysqlnyc,-14400,public,Data Enthusiasts
1,182508463.0,not_found,New York,us,New York,2017-11-08 15:15:21,40.75,http://www.meetup.com/members/182508463,-73.97,Vikas Minocha,...,https://secure.meetupstatic.com/photos/member/...,https://secure.meetupstatic.com/photos/member/...,member,4.51,NY,US/Eastern,mysqlnyc,-14400,public,Data Enthusiasts
2,14514615.0,not_found,New York,us,not_found,2011-04-17 13:36:49,40.79,http://www.meetup.com/members/14514615,-73.95,Abel M,...,https://secure.meetupstatic.com/photos/member/...,https://secure.meetupstatic.com/photos/member/...,member,4.76,NY,US/Eastern,nycvolunteerandcommunity,-14400,public,Helping Hands
3,14514615.0,not_found,New York,us,not_found,2011-04-17 13:38:51,40.79,http://www.meetup.com/members/14514615,-73.95,Abel M,...,https://secure.meetupstatic.com/photos/member/...,https://secure.meetupstatic.com/photos/member/...,member,4.37,NY,US/Eastern,Hadoop-NYC,-14400,public,Hadoop Users
4,14514615.0,not_found,New York,us,not_found,2011-04-17 14:06:05,40.79,http://www.meetup.com/members/14514615,-73.95,Abel M,...,https://secure.meetupstatic.com/photos/member/...,https://secure.meetupstatic.com/photos/member/...,member,4.72,NY,US/Eastern,music-techster,-14400,public,MusicTechsters


In [76]:
merged_data.shape

(138, 49)

In [77]:
merged_data.to_csv("other_recommendation_part1.csv")

In [79]:
fetch_cats = merged_data['category_id'].to_list()

In [80]:
fetch_cats = set(fetch_cats)
fetch_cats = list(fetch_cats)
print(fetch_cats)

[1, 2, 4, 8, 9, 10, 12, 13, 14, 16, 18, 20, 21, 22, 23, 27, 29, 30, 31, 32, 34, 36]


In [81]:
groups_new_for_demo = pd.read_csv("groups.csv", encoding = "Latin-1")
filter_groups_new_for_demo = groups_new_for_demo['category_id'].isin(fetch_cats)
groups_new_for_demo.where(filter_groups_new_for_demo,inplace=True)
groups_new_for_demo = groups_new_for_demo.dropna()
groups_new_for_demo.head()

,group_id,category_id,category.name,category.shortname,city_id,city,country,created,description,group_photo.base_url,...,organizer.photo.photo_link,organizer.photo.thumb_link,organizer.photo.type,rating,state,timezone,urlname,utc_offset,visibility,who
0,6388.0,14.0,health/wellbeing,health-wellbeing,10001.0,New York,US,2002-11-21 16:50:46,Those who practice or hold a strong interest i...,https://secure.meetupstatic.com,...,https://secure.meetupstatic.com/photos/member/...,https://secure.meetupstatic.com/photos/member/...,member,4.39,NY,US/Eastern,alternative-health-nyc,-14400.0,public,Explorers of Health
1,6510.0,4.0,community/environment,community-environment,10001.0,New York,US,2003-05-20 14:48:54,The New York Alternative Energy Meetupis for t...,https://secure.meetupstatic.com,...,https://secure.meetupstatic.com/photos/member/...,https://secure.meetupstatic.com/photos/member/...,member,4.31,NY,US/Eastern,alternative-energy-meetup,-14400.0,public,Clean Energy Supporters
3,8940.0,29.0,sci-fi/fantasy,sci-fi-fantasy,10001.0,New York,US,2002-11-16 04:49:16,Welcome to the The New York City Anime Meetup ...,https://secure.meetupstatic.com,...,https://secure.meetupstatic.com/photos/member/...,https://secure.meetupstatic.com/photos/member/...,member,4.46,NY,US/Eastern,NYC-Anime,-14400.0,public,Anime Fans
5,10359.0,16.0,language/ethnic identity,language,10001.0,New York,US,2003-05-22 14:19:52,"Hello! Michael Kim 'Jamal' Riegelman, the Dir...",https://secure.meetupstatic.com,...,https://secure.meetupstatic.com/photos/member/...,https://secure.meetupstatic.com/photos/member/...,member,4.58,NY,US/Eastern,NYCArabicClub,-14400.0,public_limited,Arabophiles!
6,12111.0,16.0,language/ethnic identity,language,10001.0,New York,US,2002-10-08 17:22:28,NYC ASL is a social/conversation group for Dea...,https://secure.meetupstatic.com,...,https://secure.meetupstatic.com/photos/member/...,https://secure.meetupstatic.com/photos/member/...,member,4.46,NY,US/Eastern,NYCASL,-14400.0,public_limited,NYC ASL Member


In [82]:
groups_new_for_demo.shape

(14146, 36)

In [83]:
groups_new_for_demo.to_csv("groups_new_for_demo.csv")

In [85]:
member_list_groups_new_for_demo = groups_new_for_demo['group_id'].to_list()

In [86]:
fetch_all_mems = pd.read_csv("members.csv", encoding = "Latin-1")
f_mem = fetch_all_mems['group_id'].isin(member_list_groups_new_for_demo)
fetch_all_mems.where(f_mem,inplace=True)
fetch_all_mems = fetch_all_mems.dropna()
fetch_all_mems.head()

,member_id,bio,city,country,hometown,joined,lat,link,lon,member_name,state,member_status,visited,group_id
0,3.0,not_found,New York,us,"New York, NY",2007-05-01 22:04:37,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2009-09-18 18:32:23,490552.0
1,3.0,not_found,New York,us,"New York, NY",2011-01-23 14:13:17,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2011-03-20 01:02:11,1474611.0
2,3.0,"Hi, I'm Matt. I'm an entrepreneur who has star...",New York,us,"New York, NY",2010-12-30 18:47:34,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2011-01-18 20:37:23,1490492.0
3,3.0,"Hi, I'm Matt. I'm an entrepreneur who has star...",New York,us,"New York, NY",2011-01-03 14:45:21,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2011-07-23 03:42:28,1515830.0
4,3.0,"Hi, I'm Matt. I'm an entrepreneur who has star...",New York,us,"New York, NY",2010-12-30 18:34:50,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2011-06-13 18:33:23,1574965.0


In [87]:
fetch_all_mems.shape

(5498113, 14)

In [88]:
fetch_all_mems.to_csv("fetch_all_mems_group_demo.csv")